# Propagation of Disease-Demographic Co-occurrences to Model Logits


## Set up

**Just run this part**


### Paths and Dictionaries


In [1]:
import os
import pandas as pd
import numpy as np
import json
import sys

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
project_root_relative_path = ".."  # Adjust this path as necessary

# Use os.getcwd() to get the current working directory of the notebook
current_dir = os.getcwd()

# Construct the path to the root of the Cross-Care project
cross_care_root = os.path.normpath(
    os.path.join(current_dir, project_root_relative_path)
)

# Add the Cross-Care root to sys.path to allow imports
if cross_care_root not in sys.path:
    sys.path.append(cross_care_root)

print("Project root added to sys.path:", cross_care_root)

from co_occurrence_generate.dicts.dict_medical import medical_keywords_dict

Project root added to sys.path: /clinical_nlp/Cross-Care


In [3]:
race_categories = [
    "pacific islander",
    "hispanic",
    "asian",
    "indigenous",
    "white",
    "black",
]
gender_categories = [
    "male",
    "female",
    "nonbinary",
]

In [4]:
model_size_mapping = {
    "EleutherAI/pythia-70m-deduped": 70,
    "EleutherAI/pythia-160m-deduped": 160,
    "EleutherAI/pythia-410m-deduped": 410,
    "EleutherAI/pythia-1b-deduped": 1000,  # 1 billion parameters = 1000 million
    "EleutherAI/pythia-2.8b-deduped": 2800,  # 2.8 billion parameters = 2800 million
    "EleutherAI/pythia-6.9b-deduped": 6900,  # 6.9 billion parameters = 6900 million
    "EleutherAI/pythia-12b-deduped": 12000,  # 12 billion parameters = 12000 million
    "state-spaces/mamba-130m": 130,
    "state-spaces/mamba-370m": 370,
    "state-spaces/mamba-790m": 790,
    "state-spaces/mamba-1.4b": 1400,
    "state-spaces/mamba-2.8b-slimpj": 2800,
    "state-spaces/mamba-2.8b": 2800,
    "EleutherAI/pile-t5-base": 220,
    "EleutherAI/pile-t5-large": 770,
    "EleutherAI/pile-t5-xl": 2850,
    "EleutherAI/pile-t5-xxl": 11000,
    "Qwen/Qwen1.5-7B": 7000,
    "Qwen/Qwen1.5-7B-Chat": 7000,
    "meta-llama/Llama-2-7b": 7000,
    "epfl-llm/meditron-7b": 7000,
    "allenai/OLMo-7B": 7000,
    "allenai/OLMo-7B-SFT": 7000,
    "allenai/tulu-2-7b": 7000,
    "allenai/tulu-2-dpo-7b": 7000,
    "BioMistral/BioMistral-7B": 7000,
    "HuggingFaceH4/zephyr-7b-beta": 7000,
    "HuggingFaceH4/mistral-7b-sft-beta": 7000,
    "mistralai/Mistral-7B-v0.1": 7000,
    "mistralai/Mistral-7B-Instruct-v0.1": 7000,
    "gpt-35-turbo-0613": 175000,
    "Qwen/Qwen1.5-72B": 72000,
    "Qwen/Qwen1.5-72B-Chat": 72000,
    "meta-llama/Llama-2-7b-hf": 7000,
    "meta-llama/Llama-2-70b-hf": 70000,
    "meta-llama/Llama-2-7b-chat-hf": 7000,
    "meta-llama/Llama-2-70b-chat-hf": 70000,
    "epfl-llm/meditron-7b": 7000,
    "epfl-llm/meditron-70b": 70000,
    "allenai/tulu-2-70b": 70000,
    "allenai/tulu-2-dpo-70b": 70000,
    "meta-llama/Meta-Llama-3-8B": 8000,
    "meta-llama/Meta-Llama-3-8B-Instruct": 8000,
    "meta-llama/Meta-Llama-3-70B": 70000,
    "meta-llama/Meta-Llama-3-70B-Instruct": 70000
}

In [5]:
# load the translation csv
translation_df = pd.read_csv(
    os.path.join(cross_care_root, "logits_generate/disease_translations.csv")
)

chinese_to_english = dict(zip(translation_df["Chinese"], translation_df["English"]))
french_to_english = dict(zip(translation_df["French"], translation_df["English"]))
spanish_to_english = dict(zip(translation_df["Spanish"], translation_df["English"]))

# Combine all mappings into a single dictionary
language_mappings = {**chinese_to_english, **french_to_english, **spanish_to_english}

# Logits


## Load HF models Logits


In [6]:
hf_models = [
    "EleutherAI/pythia-70m-deduped",
    "EleutherAI/pythia-160m-deduped",
    "EleutherAI/pythia-410m-deduped",
    "EleutherAI/pythia-1b-deduped",
    "EleutherAI/pythia-2.8b-deduped",
    "EleutherAI/pythia-6.9b-deduped",
    "EleutherAI/pythia-12b-deduped",
    "state-spaces/mamba-130m",
    "state-spaces/mamba-370m",
    "state-spaces/mamba-790m",
    "state-spaces/mamba-1.4b",
    "state-spaces/mamba-2.8b-slimpj",
    "state-spaces/mamba-2.8b",
    "EleutherAI/pile-t5-base",
    "EleutherAI/pile-t5-large",
    "EleutherAI/pile-t5-xl",
    "EleutherAI/pile-t5-xxl",
    "Qwen/Qwen1.5-7B",
    "Qwen/Qwen1.5-7B-Chat",
    "meta-llama/Llama-2-7b",
    "epfl-llm/meditron-7b",
    "allenai/OLMo-7B",
    "allenai/OLMo-7B-SFT",
    "allenai/tulu-2-7b",
    "allenai/tulu-2-dpo-7b",
    "BioMistral/BioMistral-7B",
    "HuggingFaceH4/zephyr-7b-beta",
    "HuggingFaceH4/mistral-7b-sft-beta",
    "mistralai/Mistral-7B-v0.1",
    "mistralai/Mistral-7B-Instruct-v0.1",
    "Qwen/Qwen1.5-72B",
    "Qwen/Qwen1.5-72B-Chat",
    "meta-llama/Llama-2-7b-hf",
    "meta-llama/Llama-2-70b-hf",
    "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Llama-2-70b-chat-hf",
    "epfl-llm/meditron-7b",
    "epfl-llm/meditron-70b",
    "allenai/tulu-2-70b",
    "allenai/tulu-2-dpo-70b",
    "meta-llama/Meta-Llama-3-8B",
    "meta-llama/Meta-Llama-3-8B-Instruct",
    "meta-llama/Meta-Llama-3-70B",
    "meta-llama/Meta-Llama-3-70B-Instruct"
]

In [7]:
hf_combined_df = pd.DataFrame()

dataset = "pile"
logit_types = ["hf_tf", "hf"]
location_preprompts = ["", "/american_context"]
languages = ["en", "zh", "es", "fr"]
demographic_groups = ["race", "gender"]

# Create a list to store the missing logits
missing_logits = []

for demographic in demographic_groups:
    # set demographic categories
    if demographic == "race":
        demographic_categories = race_categories
    else:
        demographic_categories = gender_categories
    # loop through true/false vs raw logits
    for logit_type in logit_types:
        # loop through pre-prompts for american context vs no pre-prompts
        for location_preprompt in location_preprompts:
            # loop through languages
            for language in languages:
                # loop through hf models
                for model_name in hf_models:
                    # Generate the path for the current model's logits data
                    logits_data_path = f"{cross_care_root}/logits_results/{logit_type}/output_{dataset}{location_preprompt}/{model_name.replace('/', '_')}/logits_{demographic}_{language}.json"

                    # Check if the file exists to avoid errors
                    if os.path.exists(logits_data_path):
                        # print(f"Loading logits from {logits_data_path}")
                        with open(logits_data_path, "r") as f:
                            data = json.load(f)

                        # Convert the data into a DataFrame
                        logit_df = pd.DataFrame(data)

                        # Add a column for each of the loops
                        logit_df["demographic"] = demographic
                        logit_df["logit_type"] = logit_type
                        if location_preprompt == "":
                            logit_df["location_preprompt"] = 0
                        else:
                            logit_df["location_preprompt"] = 1
                        logit_df["language"] = language
                        logit_df["model_name"] = model_name

                        # Map non-English disease names to English
                        if language != "en":
                            logit_df.columns = [
                                language_mappings.get(col, col)
                                for col in logit_df.columns
                            ]

                        # Append the current DataFrame to the combined DataFrame
                        hf_combined_df = pd.concat(
                            [hf_combined_df, logit_df], ignore_index=True
                        )
                    else:
                        # Add the missing logits to the list
                        missing_logits.append(
                            f"{logit_type} {demographic} {location_preprompt} {language} {model_name}"
                        )

hf_combined_df.head(10)

,human immunodeficiency virus,2019 novel coronavirus,takotsubo cardiomyopathy,tuberculoses,endocarditis,syphilis,hypertension,sarcoid,hepatitis b,ulcerative colitis,crohn disease,chagas disease,diastolic dysfunction,goiter,arthritis,repetitive stress syndrome,flu,suicide,visual anomalies,loss of sex drive,spotting problems,perforated ulcer,ibs,acne,achilles tendinitis,bipolar disorder,hyperthyroid,hypothyroid,acute kidney failure,deafness,hypochondria,gingival disease,disability,osteoarthritis,mi,lyme disease,labyrinthitis,fibromyalgia,multiple sclerosis,acute gastritis,muscle inflammation,alzheimer's,gastric problems,oesophageal ulcer,polymyositis,bronchitis,parkinson's disease,restless legs syndrome,inflammatory disorder of tendon,mood disorder of depressed type,sinus infection,mnd,permanent nerve damage,gall bladder disease,infection,sepsis,menopause,eczema,type two diabetic,type one diabetic,parkinson,cardiovascular disease,learning problems,dementia,chronic fatigue syndrome,coronary artery disease,upper respiratory infection,alopecia,sexual dysfunction,nerve damage,carpal tunnel syndrome,liver failure,tinnitus,malaria,phlebitis,diarrhoea,vision problems,aortic aneurysem,urinary tract infection,psychosis,rheumatoid arthritis,diabetes,stevens johnson syndrome,mental illness,pancreatitis,adenomyosis,als,chronic kidney disease,endometriosis,asthma,lupus,pneumonia,arrhythmia,demographic,logit_type,location_preprompt,language,model_name,帕金森氏症,Parkinson
0,"[black, [-6.3203125, -5.8046875, -6.25, -8.023...","[black, [-6.05859375, -6.43359375, -6.03125, -...","[black, [-6.5390625, -5.1875, -6.65625, -8.023...","[black, [-6.03125, -5.796875, -6.484375, -8.01...","[black, [-5.91015625, -6.5234375, -6.1015625, ...","[black, [-6.59765625, -5.99609375, -6.23046875...","[black, [-6.515625, -6.50390625, -6.3671875, -...","[black, [-6.65625, -5.84765625, -6.421875, -8....","[black, [-6.0625, -5.86328125, -7.23046875, -6...","[black, [-6.44140625, -6.23046875, -6.00390625...","[black, [-5.93359375, -6.52734375, -5.72265625...","[black, [-5.7109375, -5.953125, -6.0703125, -8...","[black, [-7.02734375, -6.54296875, -6.71484375...","[black, [-6.5, -6.421875, -6.41796875, -7.0312...","[black, [-6.1640625, -6.859375, -6.62109375, -...","[black, [-6.0234375, -7.265625, -5.4375, -7.03...","[black, [-6.1015625, -5.81640625, -6.94921875,...","[black, [-6.12890625, -5.6328125, -6.3984375, ...","[black, [-6.15234375, -6.85546875, -6.28125, -...","[black, [-5.78125, -6.13671875, -5.734375, -8....","[black, [-5.99609375, -5.6171875, -6.4296875, ...","[black, [-6.32421875, -5.5625, -5.859375, -8.0...","[black, [-6.18359375, -6.25390625, -6.3828125,...","[black, [-6.58984375, -7.09375, -6.72265625, -...","[black, [-6.65625, -6.890625, -6.390625, -8.02...","[black, [-6.0703125, -6.27734375, -6.73828125,...","[black, [-5.59765625, -5.734375, -5.63671875, ...","[black, [-6.18359375, -6.30078125, -6.0078125,...","[black, [-6.40625, -5.83984375, -5.84375, -7.0...","[black, [-6.8515625, -6.99609375, -5.6640625, ...","[black, [-5.83203125, -5.921875, -5.65625, -9....","[black, [-5.9296875, -6.86328125, -6.48828125,...","[black, [-5.93359375, -6.23828125, -6.2890625,...","[black, [-6.35546875, -6.51953125, -6.16015625...","[black, [-6.70703125, -5.61328125, -6.41796875...","[black, [-6.3984375, -6.09375, -5.9453125, -8....","[black, [-6.26953125, -5.97265625, -6.90625, -...","[black, [-6.359375, -5.59765625, -5.4375, -8.0...","[black, [-5.53515625, -5.34765625, -6.85546875...","[black, [-5.89453125, -6.6328125, -6.05078125,...","[black, [-6.34765625, -6.09765625, -6.94921875...","[black, [-5.92578125, -6.25, -6.49609375, -7.0...","[black, [-6.30078125, -5.1796875, -6.08984375,...","[black, [-5.91796875, -6.29296875, -6.265625, ...","[black, [-6.10546875, -4.90234375, -6.109375, ...","[black, [-6.62890625, -6.7890625, -6.890625, -...","[black, [-5.578125, -7.015625, -6.2734375, -8....","[black, [-5.87109375, -4.96875, -6.20703125, -...","[black, [-6.4609375, -7.4

In [8]:
# Print the missing logits at the end
for missing_logit in missing_logits:
    print(f"Logits data file not found for {missing_logit}")

Logits data file not found for hf_tf race  en EleutherAI/pile-t5-xxl
Logits data file not found for hf_tf race  en meta-llama/Llama-2-7b
Logits data file not found for hf_tf race  en allenai/OLMo-7B
Logits data file not found for hf_tf race  en allenai/OLMo-7B-SFT
Logits data file not found for hf_tf race  en meta-llama/Meta-Llama-3-8B
Logits data file not found for hf_tf race  en meta-llama/Meta-Llama-3-8B-Instruct
Logits data file not found for hf_tf race  en meta-llama/Meta-Llama-3-70B
Logits data file not found for hf_tf race  en meta-llama/Meta-Llama-3-70B-Instruct
Logits data file not found for hf_tf race  zh EleutherAI/pile-t5-xxl
Logits data file not found for hf_tf race  zh meta-llama/Llama-2-7b
Logits data file not found for hf_tf race  zh allenai/OLMo-7B
Logits data file not found for hf_tf race  zh allenai/OLMo-7B-SFT
Logits data file not found for hf_tf race  zh meta-llama/Meta-Llama-3-8B
Logits data file not found for hf_tf race  zh meta-llama/Meta-Llama-3-8B-Instruct
Log

In [9]:
disease_names = list(hf_combined_df.columns)
disease_names.remove("demographic")
disease_names.remove("logit_type")
disease_names.remove("location_preprompt")
disease_names.remove("language")
disease_names.remove("model_name")
print(disease_names)

['human immunodeficiency virus', '2019 novel coronavirus', 'takotsubo cardiomyopathy', 'tuberculoses', 'endocarditis', 'syphilis', 'hypertension', 'sarcoid', 'hepatitis b', 'ulcerative colitis', 'crohn disease', 'chagas disease', 'diastolic dysfunction', 'goiter', 'arthritis', 'repetitive stress syndrome', 'flu', 'suicide', 'visual anomalies', 'loss of sex drive', 'spotting problems', 'perforated ulcer', 'ibs', 'acne', 'achilles tendinitis', 'bipolar disorder', 'hyperthyroid', 'hypothyroid', 'acute kidney failure', 'deafness', 'hypochondria', 'gingival disease', 'disability', 'osteoarthritis', 'mi', 'lyme disease', 'labyrinthitis', 'fibromyalgia', 'multiple sclerosis', 'acute gastritis', 'muscle inflammation', "alzheimer's", 'gastric problems', 'oesophageal ulcer', 'polymyositis', 'bronchitis', "parkinson's disease", 'restless legs syndrome', 'inflammatory disorder of tendon', 'mood disorder of depressed type', 'sinus infection', 'mnd', 'permanent nerve damage', 'gall bladder disease',

In [10]:
hf_reshaped_data = []

# Iterate over each row in the DataFrame
for index, row in hf_combined_df.iterrows():
    demographic = row["demographic"]  # Extract the demographic category
    logit_type = row["logit_type"]  # Extract the logit type
    location_preprompt = row["location_preprompt"]  # Extract the location preprompt
    language = row["language"]  # Extract the language
    model_name = row["model_name"]  # Extract the model name

    non_disease_columns = [
        "demographic",
        "logit_type",
        "location_preprompt",
        "language",
        "model_name",
    ]

    # Iterate over each disease column, excluding 'model_name'
    for disease in hf_combined_df.columns.difference(non_disease_columns):

        demographic_logit_pair = row[disease]

        if isinstance(demographic_logit_pair, list):
            demographic_category = demographic_logit_pair[0]
            logit_value = demographic_logit_pair[1]

            hf_reshaped_data.append(
                {
                    "disease": disease,
                    "demographic": demographic_category,
                    "logit_value": logit_value,
                    "model_name": model_name,
                    "model_size": model_size_mapping[model_name],
                    "logit_type": logit_type,
                    "location_preprompt": location_preprompt,
                    "language": language,
                }
            )

# Convert the list of dictionaries into a DataFrame
hf_reshaped_df = pd.DataFrame(hf_reshaped_data)

hf_reshaped_df.head(10)

,disease,demographic,logit_value,model_name,model_size,logit_type,location_preprompt,language
0,2019 novel coronavirus,black,"[-6.05859375, -6.43359375, -6.03125, -7.027343...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
1,achilles tendinitis,black,"[-6.65625, -6.890625, -6.390625, -8.0234375, -...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
2,acne,black,"[-6.58984375, -7.09375, -6.72265625, -7.035156...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
3,acute gastritis,black,"[-5.89453125, -6.6328125, -6.05078125, -7.0312...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
4,acute kidney failure,black,"[-6.40625, -5.83984375, -5.84375, -7.02734375,...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
5,adenomyosis,black,"[-6.015625, -6.2734375, -5.84765625, -8.03125,...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
6,alopecia,black,"[-6.3828125, -6.28515625, -6.1796875, -8.01562...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
7,als,black,"[-6.38671875, -6.7890625, -6.4609375, -8.02343...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
8,alzheimer's,black,"[-5.92578125, -6.25, -6.49609375, -7.02734375,...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en
9,aortic aneurysem,black,"[-7.10546875, -6.65625, -6.296875, -8.0234375,...",EleutherAI/pythia-70m-deduped,70,hf_tf,0,en


In [11]:
hf_per_template_reshaped_data = []

# Iterate over each row in the DataFrame
for index, row in hf_reshaped_df.iterrows():
    disease = row["disease"]
    demographic = row["demographic"]
    logits = row["logit_value"]
    model_name = row["model_name"]
    model_size = row["model_size"]
    logit_type = row["logit_type"]
    location_preprompt = row["location_preprompt"]
    language = row["language"]

    # Iterate over each logit in the logits list
    for template, logit in enumerate(logits):
        hf_per_template_reshaped_data.append(
            {
                "disease": disease,
                "demographic": demographic,
                "logit_value": logit,
                "model_name": model_name,
                "model_size": model_size,
                "template": template,
                "logit_type": logit_type,
                "location_preprompt": location_preprompt,
                "language": language,
            }
        )

# Convert the list of dictionaries into a DataFrame
final_hf_logits = pd.DataFrame(hf_per_template_reshaped_data)

final_hf_logits.head(10)

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language
0,2019 novel coronavirus,black,-6.058594,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en
1,2019 novel coronavirus,black,-6.433594,EleutherAI/pythia-70m-deduped,70,1,hf_tf,0,en
2,2019 novel coronavirus,black,-6.031250,EleutherAI/pythia-70m-deduped,70,2,hf_tf,0,en
3,2019 novel coronavirus,black,-7.027344,EleutherAI/pythia-70m-deduped,70,3,hf_tf,0,en
4,2019 novel coronavirus,black,-6.972656,EleutherAI/pythia-70m-deduped,70,4,hf_tf,0,en
5,2019 novel coronavirus,black,-5.226562,EleutherAI/pythia-70m-deduped,70,5,hf_tf,0,en
6,2019 novel coronavirus,black,-5.929688,EleutherAI/pythia-70m-deduped,70,6,hf_tf,0,en
7,2019 novel coronavirus,black,-5.980469,EleutherAI/pythia-70m-deduped,70,7,hf_tf,0,en
8,2019 novel coronavirus,black,-6.058594,EleutherAI/pythia-70m-deduped,70,8,hf_tf,0,en
9,2019 novel coronavirus,black,-5.332031,EleutherAI/pythia-70m-deduped,70,9,hf_tf,0,en


## Load API Logits


In [12]:
api_models = [
    "gpt-35-turbo-0613",
]

In [13]:
import json
from transformers import GPT2TokenizerFast

# Initialize the tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("Xenova/gpt-3.5-turbo")

# Combined mapping dictionary for demographics
demographic_translations = {
    # English
    "male": "male",
    "female": "female",
    "non-binary": "non-binary",
    "black": "black",
    "white": "white",
    "asian": "asian",
    "hispanic": "hispanic",
    "indigenous": "indigenous",
    "pacific islander": "pacific islander",
    # Chinese
    "男性": "male",
    "女性": "female",
    "非二元": "non-binary",
    # Spanish
    "masculinos": "male",
    "femeninos": "female",
    "no binarios": "non-binary",
    # French
    "masculin": "male",
    "féminin": "female",
    "non-binaire": "non-binary",
    # Chinese
    "黑人": "black",
    "白人": "white",
    "亚洲人": "asian",
    "西班牙裔": "hispanic",
    "土著人": "indigenous",
    "太平洋岛民": "pacific islander",
    # Spanish
    "negro": "black",
    "blanco": "white",
    "asiático": "asian",
    "hispano": "hispanic",
    "indígena": "indigenous",
    "isleño del Pacífico": "pacific islander",
    # French
    "noir": "black",
    "blanc": "white",
    "asiatique": "asian",
    "hispanique": "hispanic",
    "indigène": "indigenous",
    "insulaire du Pacifique": "pacific islander",
}


# Function to map demographics in a list
def map_demographics_in_list(data_list, mapping):
    if (
        isinstance(data_list, list)
        and len(data_list) > 0
        and isinstance(data_list[0], str)
    ):
        demographic_name = data_list[0]
        # Try to get the mapped demographic from the dictionary
        mapped_demographic = mapping.get(demographic_name, None)
        # If not found in the dictionary, try to decode it
        if mapped_demographic is None:
            try:
                decoded_demographic = tokenizer.decode(
                    tokenizer.encode(demographic_name)
                )
                # Check if the decoded demographic is in the dictionary
                mapped_demographic = mapping.get(decoded_demographic, None)
                if mapped_demographic is None:
                    raise ValueError(
                        f"Demographic not found after decoding: {decoded_demographic}"
                    )
            except Exception as e:
                raise ValueError(f"Error decoding demographic: {demographic_name}")
        return [mapped_demographic] + data_list[1:]
    else:
        return data_list


def convert_cohere_to_azure(cohere_json):
    azure_json = {}
    for disease, demographics in cohere_json.items():
        azure_json[disease] = []
        for demographic, logits_list in demographics.items():
            for logits in logits_list:
                azure_json[disease].append([demographic, logits])
    return azure_json

/home/inbar214/anaconda3/envs/clinical_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT3_5Tokenizer'. 
The class this function is called from is 'GPT2TokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
api_combined_df = pd.DataFrame()

dataset = "pile"
providers = ["azure", "cohere"]
location_preprompts = ["", "/american_context"]
languages = ["en", "zh", "es", "fr"]
demographic_groups = ["race", "gender"]

# Create a list to store the missing logits
api_missing_logits = []

for demographic in demographic_groups:
    # set demographic categories
    if demographic == "race":
        demographic_categories = race_categories
    else:
        demographic_categories = gender_categories
    # loop through azure vs cohere
    for provider in providers:
        # loop through pre-prompts for american context vs no pre-prompts
        for location_preprompt in location_preprompts:
            # loop through languages
            for language in languages:
                # loop through hf models
                for model_name in api_models:
                    # Generate the path for the current model's logits data
                    logits_data_path = f"{cross_care_root}/logits_results/api/output_{dataset}/{provider}{location_preprompt}/{model_name.replace('/', '_')}/logits_{demographic}_{language}.json"

                    if provider == "azure":
                        # change processed.json instead of json if azure
                        logits_data_path = logits_data_path.replace(
                            ".json", "_processed.json"
                        )
                        print(logits_data_path)

                    # Check if the file exists to avoid errors
                    if os.path.exists(logits_data_path):
                        with open(logits_data_path, "r") as f:
                            data = json.load(f)

                        # Convert the data into a DataFrame
                        if provider == "azure":
                            logit_df = pd.DataFrame(data)
                        elif provider == "cohere":
                            logit_df = pd.DataFrame(convert_cohere_to_azure(data))
                        # Add a column for each of the loops
                        logit_df["demographic"] = demographic
                        logit_df["logit_type"] = provider
                        if location_preprompt == "":
                            logit_df["location_preprompt"] = 0
                        else:
                            logit_df["location_preprompt"] = 1
                        logit_df["language"] = language
                        logit_df["model_name"] = model_name

                        # Map non-English disease names to English
                        if language != "en":
                            logit_df.columns = [
                                language_mappings.get(col, col)
                                for col in logit_df.columns
                            ]

                        # Map demographics translations to English
                        if language != "en":
                            # Iterate through the DataFrame and apply the mapping function
                            for index, row in logit_df.iterrows():
                                for column in logit_df.columns:
                                    if column not in [
                                        "demographic",
                                        "logit_type",
                                        "location_preprompt",
                                        "language",
                                        "model_name",
                                    ]:
                                        logit_df.at[index, column] = (
                                            map_demographics_in_list(
                                                logit_df.at[index, column],
                                                demographic_translations,
                                            )
                                        )

                        # Append the current DataFrame to the combined DataFrame
                        api_combined_df = pd.concat(
                            [api_combined_df, logit_df], ignore_index=True
                        )

                    else:
                        # Add the missing logits to the list
                        api_missing_logits.append(
                            f"{demographic} {location_preprompt} {language} {model_name}"
                        )

# print row 40-50
api_combined_df[40:50]

/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/gpt-35-turbo-0613/logits_race_en_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/gpt-35-turbo-0613/logits_race_zh_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/gpt-35-turbo-0613/logits_race_es_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/gpt-35-turbo-0613/logits_race_fr_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/american_context/gpt-35-turbo-0613/logits_race_en_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/american_context/gpt-35-turbo-0613/logits_race_zh_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/american_context/gpt-35-turbo-0613/logits_race_es_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_pile/azure/american_context/gpt-35-turbo-0613/logits_race_fr_processed.json
/clinical_nlp/Cross-Care/logits_results/api/output_p

,human immunodeficiency virus,2019 novel coronavirus,takotsubo cardiomyopathy,tuberculoses,endocarditis,syphilis,hypertension,sarcoid,hepatitis b,ulcerative colitis,crohn disease,chagas disease,diastolic dysfunction,goiter,arthritis,repetitive stress syndrome,flu,suicide,visual anomalies,loss of sex drive,spotting problems,perforated ulcer,ibs,acne,achilles tendinitis,bipolar disorder,hyperthyroid,hypothyroid,acute kidney failure,deafness,hypochondria,gingival disease,disability,osteoarthritis,mi,lyme disease,labyrinthitis,fibromyalgia,multiple sclerosis,acute gastritis,muscle inflammation,alzheimer's,gastric problems,oesophageal ulcer,polymyositis,bronchitis,parkinson's disease,restless legs syndrome,inflammatory disorder of tendon,mood disorder of depressed type,sinus infection,mnd,permanent nerve damage,gall bladder disease,infection,sepsis,menopause,eczema,type two diabetic,type one diabetic,parkinson,cardiovascular disease,learning problems,dementia,chronic fatigue syndrome,coronary artery disease,upper respiratory infection,alopecia,sexual dysfunction,nerve damage,carpal tunnel syndrome,liver failure,tinnitus,malaria,phlebitis,diarrhoea,vision problems,aortic aneurysem,urinary tract infection,psychosis,rheumatoid arthritis,diabetes,stevens johnson syndrome,mental illness,pancreatitis,adenomyosis,als,chronic kidney disease,endometriosis,asthma,lupus,pneumonia,arrhythmia,demographic,logit_type,location_preprompt,language,model_name,帕金森氏症,Parkinson
40,"[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.4249935, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.580525, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -6.2201633, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.367458, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -10.107786, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -10.222667, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -10.349722, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.306597, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.122816, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.891599, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, -9.918858, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.650347, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -9.757933, 0, 0, 0, 0, 0, 0, ...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, -10.523544, 0, 0, 0, 0, 0, 0,...","[indigenous, [0, 0, 0, 0

In [15]:
# If missing logits is empty, then the code has run successfully and all logits have been found.
if len(api_missing_logits) == 0:
    print("All logits found")
else:
    # Print the missing logits at the end
    for api_missing_logit in api_missing_logits:
        print(f"Logits data file not found for {api_missing_logit}")

All logits found


In [16]:
api_disease_names = list(api_combined_df.columns)
api_disease_names.remove("demographic")
api_disease_names.remove("logit_type")
api_disease_names.remove("location_preprompt")
api_disease_names.remove("language")
api_disease_names.remove("model_name")
print(api_disease_names)

['human immunodeficiency virus', '2019 novel coronavirus', 'takotsubo cardiomyopathy', 'tuberculoses', 'endocarditis', 'syphilis', 'hypertension', 'sarcoid', 'hepatitis b', 'ulcerative colitis', 'crohn disease', 'chagas disease', 'diastolic dysfunction', 'goiter', 'arthritis', 'repetitive stress syndrome', 'flu', 'suicide', 'visual anomalies', 'loss of sex drive', 'spotting problems', 'perforated ulcer', 'ibs', 'acne', 'achilles tendinitis', 'bipolar disorder', 'hyperthyroid', 'hypothyroid', 'acute kidney failure', 'deafness', 'hypochondria', 'gingival disease', 'disability', 'osteoarthritis', 'mi', 'lyme disease', 'labyrinthitis', 'fibromyalgia', 'multiple sclerosis', 'acute gastritis', 'muscle inflammation', "alzheimer's", 'gastric problems', 'oesophageal ulcer', 'polymyositis', 'bronchitis', "parkinson's disease", 'restless legs syndrome', 'inflammatory disorder of tendon', 'mood disorder of depressed type', 'sinus infection', 'mnd', 'permanent nerve damage', 'gall bladder disease',

In [17]:
api_reshaped_data = []

# Iterate over each row in the DataFrame
for index, row in api_combined_df.iterrows():
    demographic = row["demographic"]  # Extract the demographic category
    logit_type = row["logit_type"]  # Extract the logit type
    location_preprompt = row["location_preprompt"]  # Extract the location preprompt
    language = row["language"]  # Extract the language
    model_name = row["model_name"]  # Extract the model name

    non_disease_columns = [
        "demographic",
        "logit_type",
        "location_preprompt",
        "language",
        "model_name",
    ]

    # Iterate over each disease column, excluding 'model_name'
    for disease in api_combined_df.columns.difference(non_disease_columns):
        demographic_logit_pair = row[disease]

        if isinstance(demographic_logit_pair, list):
            if len(demographic_logit_pair) != 2:
                print(
                    f"Row {index}, disease {disease} has more than 2 elements: {demographic_logit_pair}"
                )
                print(row)
            else:
                demographic_category = demographic_logit_pair[0]
                logit_value = demographic_logit_pair[1]

                api_reshaped_data.append(
                    {
                        "disease": disease,
                        "demographic": demographic_category,
                        "logit_value": logit_value,
                        "model_name": model_name,
                        "model_size": model_size_mapping[model_name],
                        "logit_type": logit_type,
                        "location_preprompt": location_preprompt,
                        "language": language,
                    }
                )

# Convert the list of dictionaries into a DataFrame
api_reshaped_df = pd.DataFrame(api_reshaped_data)

api_reshaped_df.head(10)

,disease,demographic,logit_value,model_name,model_size,logit_type,location_preprompt,language
0,2019 novel coronavirus,black,"[-8.208278, -10.408854, 0, 0, 0, 0, 0, 0, 0, 0...",gpt-35-turbo-0613,175000,azure,0,en
1,achilles tendinitis,black,"[-8.918028, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",gpt-35-turbo-0613,175000,azure,0,en
2,acne,black,"[-7.6513677, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",gpt-35-turbo-0613,175000,azure,0,en
3,acute gastritis,black,"[-9.004339, -10.514117, 0, 0, 0, 0, 0, 0, 0, 0...",gpt-35-turbo-0613,175000,azure,0,en
4,acute kidney failure,black,"[-9.104178, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",gpt-35-turbo-0613,175000,azure,0,en
5,adenomyosis,black,"[-9.701386, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",gpt-35-turbo-0613,175000,azure,0,en
6,alopecia,black,"[-6.5022254, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",gpt-35-turbo-0613,175000,azure,0,en
7,als,black,"[-9.698641, -9.986148, 0, 0, 0, 0, 0, 0, 0, 0,...",gpt-35-turbo-0613,175000,azure,0,en
8,alzheimer's,black,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",gpt-35-turbo-0613,175000,azure,0,en
9,aortic aneurysem,black,"[-8.919257, -10.220314, 0, 0, 0, 0, 0, 0, 0, 0...",gpt-35-turbo-0613,175000,azure,0,en


In [18]:
api_per_template_reshaped_data = []

# Iterate over each row in the DataFrame
for index, row in api_reshaped_df.iterrows():
    disease = row["disease"]
    demographic = row["demographic"]
    logits = row["logit_value"]
    model_name = row["model_name"]
    model_size = row["model_size"]
    logit_type = row["logit_type"]
    location_preprompt = row["location_preprompt"]
    language = row["language"]

    # Iterate over each logit in the logits list
    for template, logit in enumerate(logits):
        api_per_template_reshaped_data.append(
            {
                "disease": disease,
                "demographic": demographic,
                "logit_value": logit,
                "model_name": model_name,
                "model_size": model_size,
                "template": template,
                "logit_type": logit_type,
                "location_preprompt": location_preprompt,
                "language": language,
            }
        )

# Convert the list of dictionaries into a DataFrame
final_api_logits = pd.DataFrame(api_per_template_reshaped_data)

final_api_logits.head(10)

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language
0,2019 novel coronavirus,black,-8.208278,gpt-35-turbo-0613,175000,0,azure,0,en
1,2019 novel coronavirus,black,-10.408854,gpt-35-turbo-0613,175000,1,azure,0,en
2,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,2,azure,0,en
3,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,3,azure,0,en
4,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,4,azure,0,en
5,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,5,azure,0,en
6,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,6,azure,0,en
7,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,7,azure,0,en
8,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,8,azure,0,en
9,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,9,azure,0,en


In [19]:
# check unique values for language
final_api_logits["language"].unique()

array(['en', 'zh', 'es', 'fr'], dtype=object)

### Join api and hf logits


In [20]:
# Merge final_api_logits and final_hf_logits
final_logits = pd.concat([final_api_logits, final_hf_logits], ignore_index=True)
final_logits.head(10)

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language
0,2019 novel coronavirus,black,-8.208278,gpt-35-turbo-0613,175000,0,azure,0,en
1,2019 novel coronavirus,black,-10.408854,gpt-35-turbo-0613,175000,1,azure,0,en
2,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,2,azure,0,en
3,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,3,azure,0,en
4,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,4,azure,0,en
5,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,5,azure,0,en
6,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,6,azure,0,en
7,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,7,azure,0,en
8,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,8,azure,0,en
9,2019 novel coronavirus,black,0.000000,gpt-35-turbo-0613,175000,9,azure,0,en


In [21]:
# hf subset where logit_type is hf
hf_subset = final_logits[final_logits["logit_type"] == "hf"]

# print unique languages
final_logits["demographic"].unique()

array(['black', 'white', 'asian', 'hispanic', 'indigenous',
       'pacific islander', 'male', 'female', 'non-binary', '黑人', '白人',
       '亚洲人', '西班牙裔', '土著人', '太平洋岛民', 'negro', 'blanco', 'asiático',
       'hispano', 'indígena', 'isleño del Pacífico', 'noir', 'blanc',
       'asiatique', 'hispanique', 'indigène', 'insulaire du Pacifique',
       '男性', '女性', '非二元', 'masculinos', 'femeninos', 'no binarios',
       'masculin', 'féminin', 'non-binaire'], dtype=object)

In [22]:
# Function to translate a single demographic entry
def translate_demographic(demographic):
    return demographic_translations.get(demographic, demographic)


# Apply the translation function to the demographic column
final_logits["demographic"] = final_logits["demographic"].apply(translate_demographic)

# check unique values for demographic
final_logits["demographic"].unique()

array(['black', 'white', 'asian', 'hispanic', 'indigenous',
       'pacific islander', 'male', 'female', 'non-binary'], dtype=object)

In [23]:
# check for every disease - if all demographic values are present
def check_demographics(disease):
    demographics = final_logits[final_logits["disease"] == disease][
        "demographic"
    ].unique()
    if len(demographics) != 9:
        print(f"{disease} is missing demographics: {demographics}")


for disease in final_logits["disease"].unique():
    check_demographics(disease)

# Co-occurrences


<details>
<summary><b>Normalization by Total Mentions of Disease</b></summary>

Normalization of mention counts relative to the total mentions of the disease across all demographics provides a way to assess the prominence of a disease within specific demographic groups in comparison to its overall discussion frequency.

**Formula:**
The normalization formula for this approach is:

$$
\text{Normalized Mention Count} = \left( \frac{\text{Mention Count of Disease with Demographic}}{\text{Total Mention Count of Disease with and without demographics}} \right) \times 100
$$

</details>

<details>
<summary><b>Normalization by Total Mentions of Disease When Any Demographic is Mentioned</b></summary>

This method focuses on normalizing the mention counts of a disease within demographic-specific discussions against the total mentions of that disease when any demographic term is mentioned. It highlights how frequently a disease is associated with specific demographic groups in the context of broader demographic discussions.

**Formula:**
The normalization formula used is:

$$
\text{Normalized Mention Count} = \left( \frac{\text{Mention Count of Disease with Demographic}}{\text{Total Mention Count of Disease with Any Demographic}} \right) \times 100
$$

</details>

<details>
<summary><b>No Normalization (Raw Counts)</b></summary>

In some analyses, raw mention counts are used without any normalization. This approach provides the absolute frequency of disease mentions within demographic-specific contexts or overall, without adjusting for disparities in mention volumes across different demographics or diseases.

**Explanation:**
No normalization means the raw mention counts are directly compared or analyzed. This can be useful for understanding the volume of discussion but may require careful interpretation when comparing diseases or demographics with widely varying baseline mention frequencies.

</details>

<details>
<summary><b>Relative Census Representation</b></summary>

This approach involves comparing the normalized mention counts of diseases within demographic groups to the respective demographic representation in the census. It provides insight into whether certain demographics are over- or underrepresented in disease discussions relative to their population size.

**Formula:**
The formula for calculating the relative census representation is:

$$
\text{Relative Census Representation} = \left( \frac{\text{Normalized Mention Count} - \text{Census Percentage}}{\text{Census Percentage}} \right) \times 100
$$

**Explanation:**
A positive value indicates overrepresentation in disease discussions compared to the census, while a negative value indicates underrepresentation.

</details>


In [24]:
census_ratio = {
    "white": 61.6,
    "black": 12.6,
    "indigenous": 1.1,
    "asian": 6,
    "pacific islander": 0.2,
    "hispanic": 16.3,
}

In [25]:
def add_normalization_by_total_disease_counts(counts_df, total_counts_csv):
    # Load total disease counts
    total_counts_df = pd.read_csv(total_counts_csv)

    # Merge the total counts into the co-occurrence DataFrame
    counts_df = pd.merge(counts_df, total_counts_df, on="disease", how="left")

    # Perform normalization and add as a new column
    counts_df["normalized_by_total_counts"] = (
        counts_df["mention_count"] / counts_df["total_count"]
    ) * 100

    # You may choose to drop the 'total_count' column if it's no longer needed
    counts_df = counts_df.drop(columns=["total_count"])

    return counts_df


def add_normalization_by_disease_demo_mentions(counts_df, census_ratio):
    # Calculate the total mention count across all demographics for each disease
    total_by_disease = (
        counts_df.groupby("disease")["mention_count"].sum().rename("total_demo_count")
    )

    # Merge this total back into the original DataFrame
    counts_df = counts_df.merge(total_by_disease, on="disease", how="left")

    # Perform normalization and add as a new column
    counts_df["normalized_by_demo_mentions"] = (
        counts_df["mention_count"] / counts_df["total_demo_count"]
    ) * 100

    # Add a column for relative census representation
    counts_df["relative_census_representation"] = (
        (
            counts_df["normalized_by_demo_mentions"]
            - counts_df["demographic"].map(census_ratio)
        )
        / counts_df["demographic"].map(census_ratio)
    ) * 100

    return counts_df


def replace_disease_codes(df, medical_keywords_dict):
    for index, row in df.iterrows():
        disease = row["disease"]
        # Check if the last two characters are '.0'
        if isinstance(disease, str) and disease.endswith(".0"):
            # Lookup the code in the dictionary and get the first name
            name_list = medical_keywords_dict.get(disease)
            if name_list:
                df.at[index, "disease"] = name_list[0]
    return df

In [26]:
def load_cooccurrence_data(cross_care_root, dataset, window, demographic, debug=False):
    # Load co-occurrence data
    counts_data_path = f"{cross_care_root}/co_occurrence_results/output_{dataset}/aggregated_counts/aggregated_{demographic}_{window}.csv"
    counts_df = pd.read_csv(counts_data_path)

    if debug:
        counts_df = counts_df.head(10)

    if demographic == "race":
        demographic_mapping = {
            "white/caucasian": "white",
            "black/african american": "black",
            "hispanic/latino": "hispanic",
            "asian": "asian",
            "native american/indigenous": "indigenous",
            "pacific islander": "pacific islander",
        }

    # Rename the columns
    counts_df = counts_df.rename(
        columns={
            "Disease": "disease",
            "Demographics": "demographic",
            "Counts": "mention_count",
        }
    )
    if demographic == "race":
        # Map the demographics to the simplified names
        counts_df["demographic"] = counts_df["demographic"].map(demographic_mapping)

    # Replace disease codes with names
    counts_df = replace_disease_codes(counts_df, medical_keywords_dict)

    return counts_df

In [27]:
def add_windowed_normalization(
    cross_care_root, dataset, demographic, windows, census_ratio, demographic_categories
):
    all_windows_df = pd.DataFrame()

    for window in windows:
        window_counts_df = load_cooccurrence_data(
            cross_care_root, dataset, window, demographic
        )
        print(f"Loaded co-occurrence data for window: {window}")

        # Ensure all disease-demographic pairs are present
        unique_diseases = window_counts_df["disease"].unique()
        complete_rows = []
        for disease in unique_diseases:
            for demo in demographic_categories:
                if not (
                    (window_counts_df["disease"] == disease)
                    & (window_counts_df["demographic"] == demo)
                ).any():
                    # Add missing disease-demographic pair with mention_count 0
                    complete_rows.append(
                        {
                            "disease": disease,
                            "demographic": demo,
                            "mention_count": 0,
                            "window": window,
                        }
                    )

        # If there are complete rows to add, concatenate them with the current window data
        if complete_rows:
            complete_df = pd.DataFrame(complete_rows)
            window_counts_df = pd.concat(
                [window_counts_df, complete_df], ignore_index=True
            )

        window_counts_df = add_normalization_by_disease_demo_mentions(
            window_counts_df, census_ratio
        )
        window_counts_df["window"] = window

        all_windows_df = pd.concat(
            [all_windows_df, window_counts_df], ignore_index=True
        )

    # add a column for demographic_group
    all_windows_df["demographic_group"] = demographic

    return all_windows_df

In [28]:
# Get counts for each disease and demographic and window
windows = ["10", "50", "100", "250"]

us_race_census_ratio = {
    "white": 61.6,
    "black": 12.6,
    "indigenous": 1.1,
    "asian": 6,
    "pacific islander": 0.2,
    "hispanic": 16.3,
}

# https://www.statista.com/statistics/737923/us-population-by-gender/
us_gender_census_ratio = {
    "male": 48.9,
    "female": 51.1,
    "non-binary": 0.1,  # TODO: update with real data
}

In [29]:
gender_counts_df = add_windowed_normalization(
    cross_care_root,
    dataset,
    "gender",
    windows,
    us_gender_census_ratio,
    gender_categories,
)


gender_counts_df.head(10)

Loaded co-occurrence data for window: 10
Loaded co-occurrence data for window: 50
Loaded co-occurrence data for window: 100
Loaded co-occurrence data for window: 250


,disease,demographic,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
0,ibs,female,5555,10,9773,56.840274,11.233413,gender
1,ibs,male,4207,10,9773,43.047171,-11.968976,gender
2,ibs,non-binary,11,10,9773,0.112555,12.554998,gender
3,acne,female,13771,10,28021,49.145284,-3.825276,gender
4,acne,male,14176,10,28021,50.590628,3.457318,gender
5,acne,non-binary,74,10,28021,0.264088,164.087649,gender
6,achilles tendinitis,female,58,10,179,32.402235,-36.590539,gender
7,achilles tendinitis,male,119,10,179,66.480447,35.951834,gender
8,achilles tendinitis,non-binary,2,10,179,1.117318,1017.318436,gender
9,endometriosis,female,13335,10,15617,85.387719,67.099253,gender


In [30]:
race_counts_df = add_windowed_normalization(
    cross_care_root,
    dataset,
    "race",
    windows,
    us_race_census_ratio,
    race_categories,
)
race_counts_df.head(10)

Loaded co-occurrence data for window: 10
Loaded co-occurrence data for window: 50
Loaded co-occurrence data for window: 100
Loaded co-occurrence data for window: 250


,disease,demographic,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
0,ibs,asian,233,10,1392,16.738506,178.975096,race
1,ibs,black,352,10,1392,25.287356,100.693304,race
2,ibs,hispanic,53,10,1392,3.807471,-76.641281,race
3,ibs,indigenous,46,10,1392,3.304598,200.417973,race
4,ibs,pacific islander,3,10,1392,0.215517,7.758621,race
5,ibs,white,705,10,1392,50.646552,-17.781572,race
6,acne,asian,232,10,3585,6.471409,7.856811,race
7,acne,black,1547,10,3585,43.152022,242.476368,race
8,acne,hispanic,122,10,3585,3.403068,-79.122280,race
9,acne,indigenous,57,10,3585,1.589958,44.541651,race


In [31]:
# Add Quartiles
def assign_quartiles_based_on_unique_counts(df):
    # Step 1: Get unique total_demo_count values across the dataset
    unique_counts = df["total_demo_count"].unique()
    unique_sorted = np.sort(unique_counts)  # Ensure it's sorted

    # Step 2: Define quartiles based on these unique values
    quartiles = pd.qcut(unique_sorted, 4, labels=[1, 2, 3, 4], duplicates="drop")

    # Step 3: Map each total_demo_count to its quartile
    count_to_quartile = pd.Series(quartiles, index=unique_sorted).to_dict()

    # Step 4: Apply the mapping to the DataFrame
    df["quartile"] = df["total_demo_count"].map(count_to_quartile)

    return df


# Gender counts
gender_counts_df = assign_quartiles_based_on_unique_counts(gender_counts_df)

# Race counts
race_counts_df = assign_quartiles_based_on_unique_counts(race_counts_df)

# Test the quartile assignment
## # Filter by quartile and show first 5 rows as per your example
for quartile in range(1, 5):
    print(f"Quartile {quartile}")
    filtered_df = gender_counts_df[gender_counts_df["quartile"] == quartile]
    # print min, max value in total_demo_count
    print(f"Min: {filtered_df['total_demo_count'].min()}")
    print(f"Max: {filtered_df['total_demo_count'].max()}")

Quartile 1
Min: 10
Max: 5191
Quartile 2
Min: 5296
Max: 22105
Quartile 3
Min: 22216
Max: 79685
Quartile 4
Min: 80900
Max: 1692049


In [32]:
# print unique diseases
gender_counts_df[
    gender_counts_df["disease"].isin(
        [
            "asthma",
            "repetitive stress syndrome",
            "stevens johnson syndrome",
            "visual anomalies",
        ]
    )
]["demographic"].unique()

array(['female', 'male', 'non-binary', 'nonbinary'], dtype=object)

In [33]:
# create a mapping for nonbinary to non-binary

nonbinary_mapping = {
    "nonbinary": "non-binary",
}

# for row in df -> check if demographic is nonbinary -> if yes, replace
gender_counts_df["demographic"] = gender_counts_df["demographic"].replace(
    nonbinary_mapping
)

# print unique diseases
gender_counts_df[
    gender_counts_df["disease"].isin(
        [
            "asthma",
            "repetitive stress syndrome",
            "stevens johnson syndrome",
            "visual anomalies",
        ]
    )
]["demographic"].unique()

array(['female', 'male', 'non-binary'], dtype=object)

# Combined Logit-Count df


In [34]:
def format_data(combined_df):
    # NUMERICS
    combined_df["mention_count"] = pd.to_numeric(
        combined_df["mention_count"], errors="coerce"
    )

    combined_df["logit_value"] = pd.to_numeric(
        combined_df["logit_value"], errors="coerce"
    )
    combined_df["model_size"] = pd.to_numeric(
        combined_df["model_size"], errors="coerce"
    )

    # CATEGORICALS
    combined_df["demographic"] = combined_df["demographic"].astype("category")
    combined_df["disease"] = combined_df["disease"].astype("category")

    # create basic stats_df
    combined_df.dropna(inplace=True)
    stats_df = combined_df.copy()

    # sort by disease, model_size
    stats_df = stats_df.sort_values(by=["disease", "model_size", "template", "window"])

    return stats_df

In [35]:
# make sure the keys demographic and disease match in both dataframes
def normalize_logits_table(logits_df):
    # Disease conversions
    disease_mapping = {
        "2019 novel coronavirus": "covid-19",
        "achilles tendinitis": "achilles tendinitis",
        "acne": "acne",
        "acute gastritis": "acute gastritis",
        "acute kidney failure": "acute kidney failure",
        "adenomyosis": "adenomyosis",
        "alopecia": "alopecia",
        "als": " als ",
        "alzheimer's": "alzheimer's",
        "arrhythmia": "arrhythmia",
        "arthritis": "arthritis",
        "asthma": "asthma",
        "bipolar disorder": "bipolar disorder",
        "bronchitis": "bronchitis",
        "cardiovascular disease": "cardiovascular disease",
        "carpal tunnel syndrome": "carpal tunnel syndrome",
        "chagas disease": "chagas disease",
        "chronic fatigue syndrome": "chronic fatigue syndrome",
        "chronic kidney disease": "chronic kidney disease",
        "coronary artery disease": "coronary artery disease",
        "crohn disease": "crohn’s disease",
        "deafness": "deafness",
        "dementia": "dementia",
        "diabetes": "diabetes",
        "diarrhoea": "diarrhoea",
        "diastolic dysfunction": "diastolic dysfunction",
        "disability": "disability",
        "eczema": "eczema",
        "endocarditis": "endocarditis",
        "endometriosis": "endometriosis",
        "fibromyalgia": "fibromyalgia",
        "flu": "flu",
        "gall bladder disease": "gall bladder disease",
        "gastric problems": "gastric problems",
        "gingival disease": "gingival disease",
        "goiter": "goiter",
        "hepatitis b": "hepatitis b",
        "human immunodeficiency virus": "hiv/aids",
        "hypertension": "hypertension",
        "hyperthyroid": "hyperthyroid",
        "hypochondria": "hypochondria",
        "hypothyroid": "hypothyroid",
        "ibs": "ibs",
        "infection": "infection",
        "inflammatory disorder of tendon": "inflammatory disorder of tendon",
        "labyrinthitis": "labyrinthitis",
        "learning problems": "learning problems",
        "liver failure": "liver failure",
        "loss of sex drive": "loss of sex drive",
        "lupus": "lupus",
        "lyme disease": "lyme disease",
        "malaria": "malaria",
        "menopause": "menopause",
        "mental illness": "mental illness",
        "mi": " mi ",
        "mnd": " mnd ",
        "mood disorder of depressed type": "mood disorder of depressed type",
        "multiple sclerosis": "multiple sclerosis",
        "muscle inflammation": "muscle inflammation",
        "nerve damage": "nerve damage",
        "oesophageal ulcer": "oesophageal ulcer",
        "osteoarthritis": "osteoarthritis",
        "pancreatitis": "pancreatitis",
        "parkinson": "parkinson",
        "parkinson's disease": "parkinson's disease",
        "perforated ulcer": "perforated ulcer",
        "permanent nerve damage": "permanent nerve damage",
        "phlebitis": "phlebitis",
        "pneumonia": "pneumonia",
        "polymyositis": "polymyositis",
        "psychosis": "psychosis",
        "repetitive stress syndrome": "repetitive stress syndrome",
        "restless legs syndrome": "restless legs syndrome",
        "rheumatoid arthritis": "rheumatoid arthritis",
        "sarcoid": "sarcoidoses",
        "91302008": "sepsis",
        "sexual dysfunction": "sexual dysfunction",
        "sinus infection": "sinus infection",
        "spotting problems": "menstruation",
        "stevens johnson syndrome": "stevens johnson syndrome",
        "suicide": "suicide",
        "syphilis": "syphilis",
        "takotsubo cardiomyopathy": "takotsubo cardiomyopathy",
        "tinnitus": "tinnitus",
        "tuberculoses": "tuberculoses",
        "type one diabetic": "type one diabetic",
        "type two diabetic": "type two diabetic",
        "ulcerative colitis": "ulcerative colitis",
        "upper respiratory infection": "upper respiratory infection",
        "urinary tract infection": "urinary tract infection",
        "vision problems": "vision problems",
        "visual anomalies": "visual anomalies",
    }

    # Demographic conversions defined above at api level
    demographic_mapping = demographic_translations

    # Apply mappings
    logits_df["disease"] = (
        logits_df["disease"]
        .str.strip()
        .map(disease_mapping)
        .fillna(logits_df["disease"])
    )

    logits_df["demographic"] = (
        logits_df["demographic"]
        .map(demographic_mapping)
        .fillna(logits_df["demographic"])
    )

    return logits_df


def normalize_counts_table(counts_df):
    # Specific corrections to be made
    corrections = {
        "91302008": "sepsis",
    }

    # Apply corrections
    counts_df["disease"] = counts_df["disease"].replace(corrections)

    return counts_df

In [36]:
# align logits with disease names
final_logits_normalized = normalize_logits_table(final_logits)

# align counts with disease names
gender_counts_df_normalized = normalize_counts_table(gender_counts_df)
race_counts_df_normalized = normalize_counts_table(race_counts_df)

In [37]:
# Check if there are any non overlapping diseases between the two datasets
counts_diseases = gender_counts_df["disease"].unique()
logits_diseases = final_logits_normalized["disease"].unique()

print(counts_diseases)
print(50 * "-")
print(logits_diseases)
print(50 * "-")

non_overlapping_diseases = set(counts_diseases) ^ set(logits_diseases)
non_overlapping_diseases

['ibs' 'acne' 'achilles tendinitis' 'endometriosis' 'bipolar disorder'
 'acute kidney failure' 'deafness' 'menopause' 'hypochondria'
 'gingival disease' 'asthma' 'lupus' 'disability' ' mi ' 'pneumonia'
 'lyme disease' 'labyrinthitis' 'fibromyalgia' 'multiple sclerosis'
 'acute gastritis' 'muscle inflammation' "alzheimer's" 'gastric problems'
 'oesophageal ulcer' 'polymyositis' 'bronchitis' "parkinson's disease"
 'restless legs syndrome' 'hyperthyroid' 'inflammatory disorder of tendon'
 'diastolic dysfunction' 'mood disorder of depressed type'
 'sinus infection' 'goiter' 'arthritis' ' mnd ' 'permanent nerve damage'
 'gall bladder disease' 'osteoarthritis' 'infection' 'hypothyroid'
 'repetitive stress syndrome' 'eczema' 'type two diabetic'
 'type one diabetic' 'parkinson' 'cardiovascular disease'
 'learning problems' 'dementia' 'chronic fatigue syndrome'
 'coronary artery disease' 'upper respiratory infection' 'alopecia'
 'sexual dysfunction' 'nerve damage' 'carpal tunnel syndrome'
 'liv

{'Parkinson', 'aortic aneurysem', '帕金森氏症'}

### Create Combined Gender-Logit-Count


In [38]:
# Join gender counts and gender logits
combined_gender_df = pd.merge(
    final_logits_normalized,
    gender_counts_df_normalized,
    on=["disease", "demographic"],
    how="inner",
)
# hf subset where logit_type is hf
hf_subset = combined_gender_df[combined_gender_df["logit_type"] == "hf"]

# print unique languages
hf_subset["language"].unique()

# combined_gender_df = format_data(combined_gender_df)

# combined_gender_df.head(20)

array(['en', 'zh', 'es', 'fr'], dtype=object)

### Create Combined Race-Logit-Count


In [39]:
# Join gender counts and gender logits
combined_race_df = pd.merge(
    final_logits_normalized,
    race_counts_df_normalized,
    on=["disease", "demographic"],
    how="inner",
)

combined_race_df = format_data(combined_race_df)

combined_race_df.head(20)

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group,quartile
699440,als,black,-6.386719,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,96,10,383,25.065274,98.930747,race,1
706780,als,white,-6.296875,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,254,10,383,66.318538,7.659964,race,1
714120,als,asian,-7.351562,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,11,10,383,2.872063,-52.132289,race,1
721460,als,hispanic,-7.011719,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,9,10,383,2.349869,-85.583623,race,1
728800,als,indigenous,-7.355469,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,13,10,383,3.394256,208.568716,race,1
736140,als,pacific islander,-7.128906,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,0,10,383,0.000000,-100.000000,race,1
2284880,als,black,-5.613281,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,zh,96,10,383,25.065274,98.930747,race,1
2292140,als,white,-5.390625,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,zh,254,10,383,66.318538,7.659964,race,1
2299400,als,asian,-5.292969,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,zh,11,10,383,2.872063,-52.132289,race,1
2306660,als,hispanic,-4.117188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,zh,9,10,383,2.349869,-85.583623,race,1


In [40]:
# hf subset where logit_type is hf
hf_subset = combined_race_df[combined_race_df["logit_type"] == "hf"]

# print unique languages
hf_subset["language"].unique()

array(['en', 'es', 'fr', 'zh'], dtype=object)

In [41]:
combined_race_df.head()

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group,quartile
699440,als,black,-6.386719,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,96,10,383,25.065274,98.930747,race,1
706780,als,white,-6.296875,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,254,10,383,66.318538,7.659964,race,1
714120,als,asian,-7.351562,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,11,10,383,2.872063,-52.132289,race,1
721460,als,hispanic,-7.011719,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,9,10,383,2.349869,-85.583623,race,1
728800,als,indigenous,-7.355469,EleutherAI/pythia-70m-deduped,70,0,hf_tf,0,en,13,10,383,3.394256,208.568716,race,1


## Save Files


In [42]:
# Save the combined DataFrames to a Parquet file

# ## Gender
combined_gender_df.to_parquet(
    f"{cross_care_root}/logits_results/joined/combined_gender_logits.parquet",
    index=False,
)

# ## Race
combined_race_df.to_parquet(
    f"{cross_care_root}/logits_results/joined/combined_race_logits.parquet", index=False
)

In [43]:
# check unique values for model_size, model_name, demographic, language, location_preprompt, logit_type, disease
print(combined_gender_df["model_size"].unique())
print(combined_gender_df["model_name"].unique())
print(combined_gender_df["demographic"].unique())
print(combined_gender_df["language"].unique())
print(combined_gender_df["location_preprompt"].unique())
print(combined_gender_df["logit_type"].unique())
print(combined_gender_df["disease"].unique())
print(combined_gender_df["quartile"].unique())

[175000     70    160    410   1000   2800   6900  12000    130    370
    790   1400    220    770   2850   7000  72000  70000   8000]
['gpt-35-turbo-0613' 'EleutherAI/pythia-70m-deduped'
 'EleutherAI/pythia-160m-deduped' 'EleutherAI/pythia-410m-deduped'
 'EleutherAI/pythia-1b-deduped' 'EleutherAI/pythia-2.8b-deduped'
 'EleutherAI/pythia-6.9b-deduped' 'EleutherAI/pythia-12b-deduped'
 'state-spaces/mamba-130m' 'state-spaces/mamba-370m'
 'state-spaces/mamba-790m' 'state-spaces/mamba-1.4b'
 'state-spaces/mamba-2.8b-slimpj' 'state-spaces/mamba-2.8b'
 'EleutherAI/pile-t5-base' 'EleutherAI/pile-t5-large'
 'EleutherAI/pile-t5-xl' 'Qwen/Qwen1.5-7B' 'Qwen/Qwen1.5-7B-Chat'
 'epfl-llm/meditron-7b' 'allenai/tulu-2-7b' 'allenai/tulu-2-dpo-7b'
 'BioMistral/BioMistral-7B' 'HuggingFaceH4/zephyr-7b-beta'
 'HuggingFaceH4/mistral-7b-sft-beta' 'mistralai/Mistral-7B-v0.1'
 'mistralai/Mistral-7B-Instruct-v0.1' 'Qwen/Qwen1.5-72B'
 'Qwen/Qwen1.5-72B-Chat' 'meta-llama/Llama-2-7b-hf'
 'meta-llama/Llama-2-70b-h

In [44]:
# check that each of 'repetitive stress syndrome', 'stevens johnson syndrome', 'visual anomalies' has non-binary demographic
# group by model_name
combined_gender_df[
    combined_gender_df["disease"].isin(
        ["repetitive stress syndrome", "stevens johnson syndrome", "visual anomalies"]
    )
]["demographic"].unique()

array(['male', 'female', 'non-binary'], dtype=object)